In [18]:
import pyranges as pr
import pandas as pd
import numpy as np
import sqlite3
# import gffutils
import time
# import pybedtools
import os
from IPython import get_ipython
import ctypes
import ray
from multiprocessing import Pool

In [22]:
ray.init(ignore_reinit_error=True)

2025-04-09 13:27:13,388	INFO worker.py:1852 -- Started a local Ray instance.


Python version:,3.12.3
Ray version:,2.44.1


In [2]:
gtf_file_name = "Homo_sapiens.GRCh38.112.chr.gtf"
# gtf_file_name = "/home/U210050044/data/DH607-project/Homo_sapiens.GRCh38.112.chr.gtf"
sql_db_name = "db-human.sqlite3"
# sql_db_name = "file:humangtf?mode=memory&cache=shared"
results_file_name = "results_human.txt"
os.system("g++ -std=c++11 -shared -fPIC -o gtf_to_sql.so gtf_to_sql.cpp -lsqlite3 -pthread")
# os.system("g++ -std=c++11 -shared -fPIC -o gtf_to_sql.so gtf_to_sql.cpp -lduckdb -pthread -lstdc++")
lib = ctypes.CDLL("./gtf_to_sql.so")
# Define the function signature
lib.run_gtf_to_sql.argtypes = [
    ctypes.c_char_p,  # db_name
    ctypes.c_char_p,  # table_name
    ctypes.c_char_p,  # input_file
    ctypes.c_int,     # num_threads
]
OUTPUT_MODE = "print"
# OUTPUT_MODE = "file"

In [3]:
# This should be a module-level global so each Ray worker gets its own connection
def get_connection():
    return sqlite3.connect(sql_db_name)

In [4]:
if OUTPUT_MODE == "file":
    results_file = open(results_file_name,"w")

In [19]:
start_time = time.time()
human_gr = pr.read_gtf(gtf_file_name)
end_time = time.time()
if OUTPUT_MODE == "file":
    results_file.write(f"Finished reading into pyranges object in time {end_time-start_time}\n")
else:
    print(f"Finished reading into pyranges object in time {end_time-start_time}")

Finished reading into pyranges object in time 57.14047312736511


In [5]:
# Call the function
start_time = time.time()
lib.run_gtf_to_sql(sql_db_name.encode("utf-8"), "human".encode("utf-8"), gtf_file_name.encode("utf-8"), 8)
end_time = time.time()
if OUTPUT_MODE == "file":
    results_file.write(f"Finished writing into sql db in time {end_time-start_time}\n")
else:
    print(f"Finished writing into sql db in time {end_time-start_time}")

Time taken to read data: 5ms
Time taken to read data: 5ms
Time taken to read data: 6ms
Time taken to read data: 5ms
Time taken to read data: 5ms
Time taken to read data: 5ms
Time taken to read data: 7ms
Time taken to read data: 5ms
Time taken to process data: 245ms
Time taken to read data: 3ms
Time taken to process data: 247ms
Time taken to process data: 263ms
Time taken to read data: 2ms
Time taken to process data: 258ms
Time taken to read data: 4ms
Time taken to read data: 3ms
Time taken to process data: 268ms
Time taken to process data: 314ms
Time taken to read data: 4ms
Time taken to read data: 3ms
Time taken to process data: 445ms
Time taken to read data: 4ms
Time taken to process data: 454ms
Time taken to read data: 3ms
Time taken to process data: 257ms
Time taken to process data: 255ms
Time taken to process data: 255ms
Time taken to read data: 4ms
Time taken to process data: 255ms
Time taken to read data: 2ms
Time taken to read data: 2ms
Time taken to read data: 2ms
Time taken t

In [6]:
conn = sqlite3.connect(sql_db_name)
cur = conn.cursor()

In [ ]:
start_time = time.time()
human_gr2 = pr.PyRanges(pd.read_sql("SELECT * FROM human", conn))
end_time = time.time()
if OUTPUT_MODE == "file":
    results_file.write(f"Finished reading from sql to pyranges in time {end_time-start_time}\n")
else:
    print(f"Finished reading from sql to pyranges in time {end_time-start_time}")

In [10]:
# create chromosome strand dictionary for usage in interval queries
def create_chrom_strand_dict(chrom_strand):
    chrom, strand = chrom_strand
    self_genes_sql = pd.read_sql_query(f"SELECT * FROM human WHERE Feature = 'gene' AND Chromosome = '{chrom}' AND Strand = '{strand}'", conn)
    return self_genes_sql

def create_chrom_strand_dict_multi():
    chrom_strand_tup = pd.read_sql_query("SELECT DISTINCT Chromosome, Strand FROM human", conn)
    chrom_strand_tup = list(zip(chrom_strand_tup["Chromosome"], chrom_strand_tup["Strand"]))
    with Pool(1) as p:
        overlapping_genes_sql = p.map(create_chrom_strand_dict, chrom_strand_tup)
    chrom_strand_dict = {}
    for i in range(len(chrom_strand_tup)):
        chrom_strand_dict[chrom_strand_tup[i]] = overlapping_genes_sql[i]
    return chrom_strand_dict

In [12]:
start_time = time.time()
chrom_strand_dict = create_chrom_strand_dict_multi()
end_time = time.time()
if OUTPUT_MODE == "file":
    results_file.write(f"Finished creating Chromosome-Strand dictionary in time {end_time-start_time}\n")
else:
    print(f"Finished creating Chromosome-Strand dictionary in time {end_time-start_time}")

Finished creating Chromosome-Strand dictionary in time 1.6298942565917969


*** SIGTERM received at time=1744185275 on cpu 7 ***
PC: @     0x757a97ea504f  (unknown)  sem_trywait
    @     0x757a97e45330  242377856  (unknown)
    @           0x64da08        160  (unknown)
    @           0x58208f         32  (unknown)
    @           0x549185         80  _PyObject_MakeTpCall
    @           0x5e11e8        384  _PyEval_EvalFrameDefault
    @           0x54a9d2        176  _PyObject_Call_Prepend
    @           0x59e09f        112  (unknown)
    @           0x599b63         48  (unknown)
    @           0x549185         80  _PyObject_MakeTpCall
    @           0x5d73c9        384  _PyEval_EvalFrameDefault
    @           0x54aa9a        176  _PyObject_Call_Prepend
    @           0x59e09f        112  (unknown)
    @           0x599b63         48  (unknown)
    @           0x54924e         80  _PyObject_MakeTpCall
    @           0x5d73c9        384  _PyEval_EvalFrameDefault
    @           0x5d58eb        128  PyEval_EvalCode
    @           0x5d347c        112 

# Aggregate Queries: Counting transcripts, calculating exon lengths, and grouping features
### Aggregate Query 1: Count the number of exons for each gene

In [ ]:
# Using pyranges
time = get_ipython().run_line_magic("timeit",'-o (human_gr[human_gr.Feature == "exon"].df.groupby("gene_id").size().reset_index(name="exon_count"))')
if OUTPUT_MODE == "file":
    results_file.write(f"Time taken to count exons using pyranges is {time}\n")
else:
    print(f"Time taken to count exons using pyranges is {time}")

In [ ]:
# Using sql
time = get_ipython().run_line_magic("timeit",'-o exon_counts_sql = pd.read_sql_query("SELECT gene_id, COUNT(*) as exon_count FROM human WHERE Feature = \'exon\' GROUP BY gene_id", conn)')
if OUTPUT_MODE == "file":
    results_file.write(f"Time taken to count exons using sql is {time}\n")
else:
    print(f"Time taken to count exons using sql is {time}")

In [ ]:
# check if exon_counts_pr and exon_counts_sql are identical
exon_counts_pr = (human_gr[human_gr.Feature == "exon"].df.groupby("gene_id").size().reset_index(name="exon_count"))
exon_counts_sql = pd.read_sql_query("SELECT gene_id, COUNT(*) as exon_count FROM human WHERE Feature = 'exon' GROUP BY gene_id", conn)
exon_counts_sql["exon_count"] = exon_counts_sql["exon_count"].astype(int)
exon_counts_pr["exon_count"] = exon_counts_pr["exon_count"].astype(int)
if exon_counts_sql.equals(exon_counts_pr):
    result_text = "exon counts are identical"
else:
    result_text = "exon counts are not identical"
if OUTPUT_MODE == "file":
    results_file.write(f"{result_text}\n")
else:
    print(result_text)

### Aggregate Query 2: Calculate the total length of exons for each gene

In [ ]:
# Using pyranges
time = get_ipython().run_line_magic("timeit",'-o exon_lengths_pr = (human_gr[human_gr.Feature == "exon"].df.assign(length=lambda df: df["End"] - df["Start"]).groupby("gene_id")["length"].sum().reset_index(name="total_exon_length"))')
if OUTPUT_MODE == "file":
    results_file.write(f"Time taken to calculate total exon length using pyranges is {time}\n")
else:
    print(f"Time taken to calculate total exon length using pyranges is {time}")

In [ ]:
# using sql
time = get_ipython().run_line_magic("timeit",'-o exon_lengths_sql = pd.read_sql_query("SELECT gene_id, SUM(End - Start) as total_exon_length FROM human WHERE Feature = \'exon\' GROUP BY gene_id", conn)')
if OUTPUT_MODE == "file":
    results_file.write(f"Time taken to calculate total exon length using sql is {time}\n")
else:
    print(f"Time taken to calculate total exon length using sql is {time}")

In [ ]:
# check if exon_lengths_pr and exon_lengths_sql are identical
exon_lengths_pr = (human_gr[human_gr.Feature == "exon"].df.assign(length=lambda df: df["End"] - df["Start"]).groupby("gene_id")["length"].sum().reset_index(name="total_exon_length"))
exon_lengths_sql = pd.read_sql_query("SELECT gene_id, SUM(End - Start) as total_exon_length FROM human WHERE Feature = 'exon' GROUP BY gene_id", conn)
exon_lengths_sql["total_exon_length"] = exon_lengths_sql["total_exon_length"].astype(int)
exon_lengths_pr["total_exon_length"] = exon_lengths_pr["total_exon_length"].astype(int)
if exon_lengths_sql.equals(exon_lengths_pr):
    results_text = "exon lengths are identical"
else:
    results_text = "exon lengths are not identical"
if OUTPUT_MODE == "file":
    results_file.write(f"{results_text}\n")
else:
    print(results_text)

### Aggregate Query 3: Identify the chromosome with the highest number of transcripts

In [ ]:
# Using pyranges
time = get_ipython().run_line_magic("timeit",'-o transcript_counts_pr = human_gr[human_gr.Feature == "transcript"].df["Chromosome"].value_counts().idxmax()')
if OUTPUT_MODE == "file":
    results_file.write(f"Time taken to find chromosome with most transcripts using pyranges is {time}\n")
else:
    print(f"Time taken to find chromosome with most transcripts using pyranges is {time}")

In [ ]:
# using sql
time = get_ipython().run_line_magic("timeit",'-o transcript_counts_sql = pd.read_sql_query("SELECT Chromosome, COUNT(*) as transcript_count FROM human WHERE Feature = \'transcript\' GROUP BY Chromosome ORDER BY transcript_count DESC LIMIT 1", conn)')
if OUTPUT_MODE == "file":
    results_file.write(f"Time taken to find chromosome with most transcripts using sql is {time}\n")
else:
    print(f"Time taken to find chromosome with most transcripts using sql is {time}")

In [ ]:
# verify if transcript_counts_pr and transcript_counts_sql are identical
transcript_counts_pr = human_gr[human_gr.Feature == "transcript"].df["Chromosome"].value_counts().idxmax()
transcript_counts_sql = pd.read_sql_query("SELECT Chromosome, COUNT(*) as transcript_count FROM human WHERE Feature = 'transcript' GROUP BY Chromosome ORDER BY transcript_count DESC LIMIT 1", conn)
if transcript_counts_pr == transcript_counts_sql["Chromosome"].values[0]:
    results_text = "transcript counts are identical"
else:
    results_text = "transcript counts are not identical"
if OUTPUT_MODE == "file":
    results_file.write(f"{results_text}\n")
else:
    print(results_text)

# Interval Arithmetic
### Query 1: Merging Overlapping Exon Intervals

In [ ]:
# Using pyranges
time = get_ipython().run_line_magic("timeit",'-o exon_intervals_pr = human_gr[human_gr.Feature == "exon"].merge(strand=True)')
if OUTPUT_MODE == "file":
    results_file.write(f"Time taken to merge exons using pyranges is {time}\n")
else:
    print(f"Time taken to merge exons using pyranges is {time}")

In [ ]:
# using sql
def get_exon_intervals_sql(chrom_strand):
    chrom, strand = chrom_strand
    exon_intervals_sql = pd.read_sql_query(f"SELECT Start, End FROM human WHERE Feature = 'exon' AND Chromosome = '{chrom}' AND Strand = '{strand}'", conn)
    exon_intervals_sql = pr.methods.merge._merge(exon_intervals_sql, chromosome=chrom, count=None, strand=strand)
    return exon_intervals_sql

def get_exon_intervals_sql_multi():
    chrom_strand_tup = pd.read_sql_query("SELECT DISTINCT Chromosome, Strand FROM human", conn)
    chrom_strand_tup = list(zip(chrom_strand_tup["Chromosome"], chrom_strand_tup["Strand"]))
    with Pool(8) as p:
        exon_intervals_sql = p.map(get_exon_intervals_sql, chrom_strand_tup)
    exon_intervals_sql = pd.concat(exon_intervals_sql).sort_values(["Chromosome", "Strand", "Start", "End"]).reset_index(drop=True)
    return exon_intervals_sql

In [ ]:
time = get_ipython().run_line_magic("timeit",'-o exon_intervals_sql = get_exon_intervals_sql_multi()')
if OUTPUT_MODE == "file":
    results_file.write(f"Time taken to merge exons using sql is {time}\n")
else:
    print(f"Time taken to merge exons using sql is {time}")

In [ ]:
# check if exon_intervals_pr and exon_intervals_sql are identical
exon_intervals_pr = human_gr[human_gr.Feature == "exon"].merge(strand=True).df.sort_values(["Chromosome", "Strand", "Start", "End"]).reset_index(drop=True)
exon_intervals_sql = get_exon_intervals_sql_multi()
if exon_intervals_pr.equals(exon_intervals_sql):
    results_text = "exon intervals are identical"
else:
    results_text = "exon intervals are not identical"
if OUTPUT_MODE == "file":
    results_file.write(f"{results_text}\n")
else:
    print(results_text)

### Query 2: Finding Overlaps with a Specific Interval
Find all gene features that overlap a given interval chr1:1000000-2000000 + strand

In [20]:
# Using pyranges
time = get_ipython().run_line_magic("timeit",'-o overlapping_genes_pr = human_gr[human_gr.Feature == "gene"].overlap(pr.from_dict({"Chromosome": ["1"], "Start": [1000000], "End": [2000000], "Strand": ["+"]}), strandedness="same")')
if OUTPUT_MODE == "file":
    results_file.write(f"Time taken to find overlapping genes using pyranges is {time}\n")
else:
    print(f"Time taken to find overlapping genes using pyranges is {time}")

387 ms ± 10.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Time taken to find overlapping genes using pyranges is 387 ms ± 10.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# # using sql
# other_genes_all = pd.DataFrame({"Chromosome": ["1"], "Start": [1000000], "End": [2000000], "Strand": ["+"]})
# def get_overlapping_genes_sql(chrom_strand):
#     chrom, strand = chrom_strand
#     self_genes_sql = pd.read_sql_query(f"SELECT * FROM human WHERE Feature = 'gene' AND Chromosome = '{chrom}' AND Strand = '{strand}'", conn)
#     other_genes = other_genes_all[(other_genes_all["Chromosome"] == chrom) & (other_genes_all["Strand"] == strand)]
#     overlapping_genes_sql = pr.methods.intersection._overlap(self_genes_sql, other_genes, how="first")
#     return overlapping_genes_sql

# def get_overlapping_genes_sql_multi():
#     chrom_strand_tup = pd.read_sql_query("SELECT DISTINCT Chromosome, Strand FROM human", conn)
#     chrom_strand_tup = list(zip(chrom_strand_tup["Chromosome"], chrom_strand_tup["Strand"]))
#     with Pool(8) as p:
#         overlapping_genes_sql = p.map(get_overlapping_genes_sql, chrom_strand_tup)
#     overlapping_genes_sql = pd.concat(overlapping_genes_sql) #.sort_values(["Chromosome", "Strand", "Start", "End"]).reset_index(drop=True)
#     return overlapping_genes_sql

In [ ]:
# # using sql
# other_genes_all = pd.DataFrame({"Chromosome": ["1"], "Start": [1000000], "End": [2000000], "Strand": ["+"]})
# def get_overlapping_genes_sql(chrom_strand):
#     chrom, strand = chrom_strand
#     self_genes_sql = pd.read_sql_query(f"SELECT * FROM human WHERE Feature = 'gene' AND Chromosome = '{chrom}' AND Strand = '{strand}'", conn)
#     # self_genes_sql = chrom_strand_dict[(chrom, strand)]
#     other_genes = other_genes_all[(other_genes_all["Chromosome"] == chrom) & (other_genes_all["Strand"] == strand)]
#     overlapping_genes_sql = pr.methods.intersection._overlap(self_genes_sql, other_genes, how="first")
#     return overlapping_genes_sql

# def get_overlapping_genes_sql_multi():
#     with Pool(1) as p:
#         overlapping_genes_sql = p.map(get_overlapping_genes_sql, chrom_strand_dict.keys())
#     overlapping_genes_sql = pd.concat(overlapping_genes_sql) #.sort_values(["Chromosome", "Strand", "Start", "End"]).reset_index(drop=True)
#     return overlapping_genes_sql

In [7]:
other_genes_all = pd.DataFrame({"Chromosome": ["1"], "Start": [1000000], "End": [2000000], "Strand": ["+"]})

@ray.remote
def get_overlapping_genes_sql(chrom_strand):
    chrom, strand = chrom_strand
    conn = get_connection()
    self_genes_sql = pd.read_sql_query(
        f"SELECT * FROM human WHERE Feature = 'gene' AND Chromosome = '{chrom}' AND Strand = '{strand}'", conn
    )
    conn.close()
    # self_genes_sql = chrom_strand_dict[(chrom, strand)]

    other_genes = other_genes_all[
        (other_genes_all["Chromosome"] == chrom) & (other_genes_all["Strand"] == strand)
    ]
    overlapping_genes_sql = pr.methods.intersection._overlap(self_genes_sql, other_genes, how="first")
    return overlapping_genes_sql

def get_overlapping_genes_sql_multi():
    futures = [get_overlapping_genes_sql.remote(chrom_strand) for chrom_strand in chrom_strand_dict.keys()]
    overlapping_genes_sql_list = ray.get(futures)
    return pd.concat(overlapping_genes_sql_list)

In [ ]:
overlapping_genes_sql = get_overlapping_genes_sql_multi()

2025-04-09 13:25:00,823	INFO worker.py:1684 -- Calling ray.init() again after it has already been called.


In [ ]:
time = get_ipython().run_line_magic("timeit",'-o overlapping_genes_sql = get_overlapping_genes_sql_multi()')
if OUTPUT_MODE == "file":
    results_file.write(f"Time taken to find overlapping genes using sql is {time}\n")
else:
    print(f"Time taken to find overlapping genes using sql is {time}")

2025-04-09 13:25:08,414	INFO worker.py:1852 -- Started a local Ray instance.


300 ms ± 5.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Time taken to find overlapping genes using sql is 300 ms ± 5.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# check if overlapping_genes_pr and overlapping_genes_sql are identical
overlapping_genes_pr = human_gr[human_gr.Feature == "gene"].overlap(pr.from_dict({"Chromosome": ["1"], "Start": [1000000], "End": [2000000], "Strand": ["+"]}), strandedness="same")
overlapping_genes_sql = get_overlapping_genes_sql_multi()
overlapping_genes_sql = overlapping_genes_sql.sort_values(["Chromosome", "Strand", "Start", "End"]).reset_index(drop=True)
overlapping_genes_pr = overlapping_genes_pr.df.sort_values(["Chromosome", "Strand", "Start", "End"]).reset_index(drop=True)
diff = overlapping_genes_pr.compare(overlapping_genes_sql)
if diff.empty:
    results_text = "overlapping genes are identical"
else:
    results_text = "overlapping genes are not identical"
if OUTPUT_MODE == "file":
    results_file.write(f"{results_text}\n")
else:
    print(results_text)

2025-04-09 13:26:42,542	INFO worker.py:1852 -- Started a local Ray instance.


overlapping genes are identical


### Query 3: Subtracting Intervals
Remove a set of repetitive regions from the exon features.

chr1:15M-16M,20M-21M

In [ ]:
other_cdf = pr.from_dict({"Chromosome": ["1", "1"],"Start": [15000000, 20000000],"End": [16000000, 21000000], "Strand": ["+", "-"]})

In [ ]:
# using pyranges
time = get_ipython().run_line_magic("timeit",'-o subtracted_intervals_pr = human_gr[human_gr.Feature == "exon"].subtract(other_cdf, strandedness="same")')
if OUTPUT_MODE == "file":
    results_file.write(f"Time taken to subtract intervals using pyranges is {time}\n")
else:
    print(f"Time taken to subtract intervals using pyranges is {time}")

In [ ]:
# using sql
def get_subtracted_intervals_sql(chrom_strand):
    chrom, strand = chrom_strand
    self_genes_sql = pd.read_sql_query(f"SELECT * FROM human WHERE Feature = 'exon' AND Chromosome = '{chrom}' AND Strand = '{strand}'", conn)
    other_cdf_clusters = other_cdf.merge(strand=True)
    other_chrom_clusters = other_cdf_clusters[(other_cdf_clusters.Chromosome == chrom) & (other_cdf_clusters.Strand == strand)].df
    # add __num__ column to self_genes_sql which counts how many intervals in self_genes_sql overlap with other_genes
    self_genes_sql = pr.methods.coverage._number_overlapping(self_genes_sql, other_chrom_clusters, strandedness="same", keep_nonoverlapping=True, overlap_col="__num__")
    subtracted_intervals_sql = pr.methods.subtraction._subtraction(self_genes_sql, other_chrom_clusters, strandedness="same")
    return subtracted_intervals_sql.drop(columns=["__num__"])

def get_subtracted_intervals_sql_multi():
    chrom_strand_tup = pd.read_sql_query("SELECT DISTINCT Chromosome, Strand FROM human", conn)
    chrom_strand_tup = list(zip(chrom_strand_tup["Chromosome"], chrom_strand_tup["Strand"]))
    with Pool(8) as p:
        subtracted_intervals_sql = p.map(get_subtracted_intervals_sql, chrom_strand_tup)
    subtracted_intervals_sql = pd.concat(subtracted_intervals_sql) #.sort_values(["Chromosome", "Strand", "Start", "End"]).reset_index(drop=True)
    return subtracted_intervals_sql

In [ ]:
time = get_ipython().run_line_magic("timeit",'-o subtracted_intervals_sql = get_subtracted_intervals_sql_multi()')
if OUTPUT_MODE == "file":
    results_file.write(f"Time taken to subtract intervals using sql is {time}\n")
else:
    print(f"Time taken to subtract intervals using sql is {time}")

In [ ]:
# check if subtracted_intervals_pr and subtracted_intervals_sql are identical
subtracted_intervals_pr = human_gr[human_gr.Feature == "exon"].subtract(other_cdf, strandedness="same")
subtracted_intervals_sql = get_subtracted_intervals_sql_multi()
subtracted_intervals_sql.to_csv("subtracted_intervals_sql.csv", index=False)
subtracted_intervals_pr.df.to_csv("subtracted_intervals_pr.csv", index=False)
os.system("cat subtracted_intervals_sql.csv | sort > subtracted_intervals_sql_sorted.csv")
os.system("cat subtracted_intervals_pr.csv | sort > subtracted_intervals_pr_sorted.csv")
diff = os.popen("diff subtracted_intervals_sql_sorted.csv subtracted_intervals_pr_sorted.csv").read()
if diff == "":
    results_text = "subtracted intervals are identical"
else:
    results_text = "subtracted intervals are not identical:" + diff
if OUTPUT_MODE == "file":
    results_file.write(f"{results_text}\n")
else:
    print(results_text)

In [ ]:
results_file.close()
conn.close()

NameError: name 'results_file' is not defined

In [24]:
ray.shutdown()